<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/SPARK_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

In [120]:
file_download_link = "https://drive.google.com/u/0/uc?id=1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi&export=download" 
!wget -O paper_graph_adj_list.txt --no-check-certificate "$file_download_link"

file_download_link = "https://drive.google.com/u/0/uc?id=1qmwKYipNwJZyheNfVxW8GwzhbY6J2nPU&export=download" 
!wget -O paper_graph_node_list.txt --no-check-certificate "$file_download_link"

--2021-04-25 11:07:51--  https://drive.google.com/u/0/uc?id=1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi&export=download
Resolving drive.google.com (drive.google.com)... 173.194.193.113, 173.194.193.102, 173.194.193.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.193.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qj71p43ejqtfpev0mlio6d3vc7301ehu/1619348850000/14577798666193718332/*/1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi?e=download [following]
--2021-04-25 11:07:51--  https://doc-0s-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qj71p43ejqtfpev0mlio6d3vc7301ehu/1619348850000/14577798666193718332/*/1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi?e=download
Resolving doc-0s-70-docs.googleusercontent.com (doc-0s-70-docs.googleusercontent.com)... 172.217.219.132, 2607:f8b0:4001:c13::84
Connecting to doc-0s-70-docs.googleusercontent.co

In [121]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [122]:
df = spark.sparkContext.textFile("paper_graph_adj_list.txt")
nodes = spark.sparkContext.textFile("paper_graph_node_list.txt")

In [123]:
temp_var = df.map(lambda k: k.split(" "))
temp_nodes = nodes.map(lambda k: k.split(" "))

In [124]:
temp_nodes.collect()

[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']]

In [140]:
def create_all_links(all_data):

  result = []
  cnt = len(all_data)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      if (all_data[i] < all_data[j]):
        result.append( ((all_data[i], all_data[j]), 0))
      else:
        result.append( ((all_data[j], all_data[i]), 0))

  return result

In [171]:
all_links = temp_nodes.flatMap(lambda k : create_all_links(k))

In [172]:
all_links.collect()

[(('A', 'B'), 0),
 (('A', 'C'), 0),
 (('A', 'D'), 0),
 (('A', 'E'), 0),
 (('A', 'F'), 0),
 (('A', 'G'), 0),
 (('A', 'H'), 0),
 (('A', 'I'), 0),
 (('A', 'J'), 0),
 (('A', 'K'), 0),
 (('B', 'C'), 0),
 (('B', 'D'), 0),
 (('B', 'E'), 0),
 (('B', 'F'), 0),
 (('B', 'G'), 0),
 (('B', 'H'), 0),
 (('B', 'I'), 0),
 (('B', 'J'), 0),
 (('B', 'K'), 0),
 (('C', 'D'), 0),
 (('C', 'E'), 0),
 (('C', 'F'), 0),
 (('C', 'G'), 0),
 (('C', 'H'), 0),
 (('C', 'I'), 0),
 (('C', 'J'), 0),
 (('C', 'K'), 0),
 (('D', 'E'), 0),
 (('D', 'F'), 0),
 (('D', 'G'), 0),
 (('D', 'H'), 0),
 (('D', 'I'), 0),
 (('D', 'J'), 0),
 (('D', 'K'), 0),
 (('E', 'F'), 0),
 (('E', 'G'), 0),
 (('E', 'H'), 0),
 (('E', 'I'), 0),
 (('E', 'J'), 0),
 (('E', 'K'), 0),
 (('F', 'G'), 0),
 (('F', 'H'), 0),
 (('F', 'I'), 0),
 (('F', 'J'), 0),
 (('F', 'K'), 0),
 (('G', 'H'), 0),
 (('G', 'I'), 0),
 (('G', 'J'), 0),
 (('G', 'K'), 0),
 (('H', 'I'), 0),
 (('H', 'J'), 0),
 (('H', 'K'), 0),
 (('I', 'J'), 0),
 (('I', 'K'), 0),
 (('J', 'K'), 0)]

In [125]:
data = temp_var.map(lambda k : (k[0], k[1::]))

In [126]:
data.collect()

[('A', ['C', 'D', 'F']),
 ('B', ['C', 'D', 'E']),
 ('C', ['A', 'B', 'F', 'D', 'G']),
 ('D', ['A', 'B', 'C', 'E']),
 ('E', ['B', 'D', 'J', 'K']),
 ('F', ['A', 'C', 'G', 'H', 'I']),
 ('G', ['C', 'F']),
 ('H', ['F', 'I']),
 ('I', ['F', 'H']),
 ('J', ['E', 'K']),
 ('K', ['E', 'J'])]

In [127]:
def calc_cc(key, value):

  result = []
  cnt = len(value)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      if (value[i] < value[j]):
        result.append( ((value[i], value[j]), key))
      else:
        result.append( ((value[j], value[i]), key))

  return result

In [128]:
cc_reduce = data.flatMap(lambda x: calc_cc(x[0], x[1]))

In [129]:
data_cn = cc_reduce.groupByKey().map(lambda x : (x[0], len(list(x[1]))))

In [130]:
def calc_car(key, value):

  result = []

  cnt = len(value)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      for k in range(j +1, cnt):
        
        if (key < value[k]):
          first_data = (key, value[k])
        else:
          first_data = (value[k], key)

        if (value[i] < value[j]):
          second_data = (value[i], value[j])
        else:
          second_data = (value[j], value[i])
    
        result.append( (second_data , (first_data, 1)))

        if (key < value[j]):
          first_data = (key, value[j])
        else:
          first_data = (value[j], key)

        if (value[i] < value[k]):
          second_data = (value[i], value[k])
        else:
          second_data = (value[k], value[i])
    
        result.append( (second_data , (first_data, 1)))
        
        if (key < value[i]):
          first_data = (key, value[i])
        else:
          first_data = (value[i], key)

        if (value[k] < value[j]):
          second_data = (value[k], value[j])
        else:
          second_data = (value[j], value[k])
    
        result.append( (second_data , (first_data, 1)))
  return result

In [131]:
car_reduce = data.flatMap(lambda x: calc_car(x[0], x[1]))

In [132]:
grouped = car_reduce.groupByKey().map(lambda x : (x[0], list(x[1])))

In [133]:
def Convert(lst):
  result = {}

  for item in lst:
    if (item[0] in result):
      result[item[0]] = result[item[0]] + 1
    else:
      result[item[0]] = 1

  return result

In [134]:
def remove_only_one_time_appers(dic):
  return sum(y / 2 for x,y in dic.items() if y> 1)

In [135]:
def car_grouped_mix(key, value):
  aggrigated = remove_only_one_time_appers(Convert(value))
  return (key, aggrigated)

In [136]:
car_grouped_reduce = grouped.map(lambda x: (car_grouped_mix(x[0], x[1])))

In [174]:
joined_data = car_grouped_reduce.union(all_links)

In [176]:
join_by_all_links = joined_data.reduceByKey(lambda x, y: x + y)

In [177]:
car_for_all_links = join_by_all_links.union(data_cn).reduceByKey(lambda x, y: x * y)

In [178]:
car_for_all_links.collect()

[(('A', 'B'), 2.0),
 (('A', 'F'), 0),
 (('D', 'G'), 0),
 (('G', 'I'), 0),
 (('D', 'F'), 2.0),
 (('B', 'D'), 0),
 (('F', 'G'), 0),
 (('B', 'E'), 0),
 (('G', 'H'), 0),
 (('E', 'F'), 0),
 (('E', 'H'), 0),
 (('C', 'D'), 0),
 (('D', 'J'), 0),
 (('D', 'K'), 0),
 (('C', 'G'), 0),
 (('C', 'I'), 0),
 (('G', 'K'), 0),
 (('C', 'H'), 0),
 (('H', 'J'), 0),
 (('H', 'K'), 0),
 (('D', 'E'), 0),
 (('B', 'H'), 0),
 (('A', 'D'), 0),
 (('B', 'G'), 0),
 (('H', 'I'), 0),
 (('D', 'H'), 0),
 (('F', 'I'), 0),
 (('A', 'C'), 0),
 (('B', 'C'), 0),
 (('E', 'J'), 0),
 (('I', 'J'), 0),
 (('B', 'J'), 0),
 (('B', 'K'), 0),
 (('A', 'K'), 0),
 (('F', 'K'), 0),
 (('B', 'F'), 0),
 (('A', 'H'), 0),
 (('C', 'J'), 0),
 (('C', 'K'), 0),
 (('D', 'I'), 0),
 (('E', 'G'), 0),
 (('E', 'I'), 0),
 (('F', 'H'), 0),
 (('A', 'G'), 2.0),
 (('A', 'E'), 0),
 (('J', 'K'), 0),
 (('A', 'I'), 0),
 (('B', 'I'), 0),
 (('C', 'E'), 2.0),
 (('E', 'K'), 0),
 (('G', 'J'), 0),
 (('I', 'K'), 0),
 (('C', 'F'), 0),
 (('A', 'J'), 0),
 (('F', 'J'), 0)]